In [ ]:
from langchain_groq import ChatGroq
import os
import pandas as pd
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, ToolMessage
from datetime import date

# Initialize the LLM
llm = ChatGroq(groq_api_key="your groq api key", model='llama3-70b-8192')


In [ ]:
@tool
def calculate( column_names,column_type ):
    '''
    Read data from an Excel sheet or DataFrame, calculate specified columns,
    and create a new column with the respective values.
    Like sum for addition of the column values, multiply for multiplication of the column values, subtraction for subtraction of the column values, division for division of the column values and power for raising to a certain power and average or mean for average of the column values.
    Parameters:
    column_names (list[str]): List of column names to be calculated.
    column_type (str): Type of calculation to be performed. Can be one of'sum','multiply', 'division', 'power', or'mean'.
    Returns:
    pd.DataFrame: DataFrame with the new column 'respective' added.
    '''
    m=1
    s=0
    a=1 
    x=0
    d=1
    file_path= "DATASHEET.xlsx"
    dob_column = "DOB"
    df = pd.read_excel(file_path)
    reference_year = date.today().year  # Get the current year
   
    
   

    if column_type == 'multiply':
        for col in column_names:
            m *= df[col]
        df['multiplied'] = m
    elif column_type =='sum':
        for col in column_names:
            s += df[col]
        df['Sum']  = s

    elif column_type == 'division':
        d = df[column_names[0]] /df[column_names[1]]
        df['division'] = d
    elif column_type == 'subtraction':
        z = df[column_names[0]]-df[column_names[1]]
        df['subtracted'] = z

    elif column_type =='mean':
        for col in column_names:
            s += df[col]
            x=x+1
        df['Average']  = s/x
    return df
    
   

In [ ]:
from datetime import date

def call_functions(user_prompt):
    system_prompt = 'You are a helpful database assistant that analyzes data and performs functions. Today is {today}'.format(today=date.today())
    
    file_path= "DATASHEET.xlsx"
    dob_column = "DOB"
    df = pd.read_excel(file_path)
    reference_year = date.today().year  # Get the current year
    
    messages = [SystemMessage(system_prompt), user_prompt]
    tools = [ calculate]  # Assuming these are your tool functions
    llm_with_tools = llm.bind_tools(tools)
    
    ai_msg = llm_with_tools.invoke(messages)
    messages.append(ai_msg)
    
    t = []
    for tool_call in ai_msg.tool_calls:
        selected_tool = { "calculate": calculate}[tool_call["name"].lower()]
        tool_output = selected_tool.invoke(tool_call["args"])
        t.append(tool_output)
    
    return t

In [ ]:
system_prompt = 'You are a helpful database assistant that analyzes data and performs functions. Today is {today}'.format(today = date.today())
HumanMessage="Calculate the age and then Give me the sum and multiplication and average of columns A B and age, the divsion and subtraction of columns A and B "
out = call_functions(HumanMessage)
print(out)